!['Problem'](HW_01.png)

In [1]:
import pandas as pd
import random
import numpy as np

# Get Data

In [2]:
use_given_data = True

if use_given_data:
    df = pd.read_csv('processing_times.csv')
else:
    # you want random data
    processing_times = []
    due_dates = []
    for i in range(12):
        x = random.choice(range(5,51))
        processing_times.append(x)
        due_dates.append(int(x * np.random.uniform(1.6, 3.8, size=1)[0]))
    data = {'JOB':range(12),'PROCESSING_TIME':processing_times,'DUE_DATE':due_dates}
    df = pd.DataFrame(data, columns = ['JOB', 'PROCESSING_TIME', 'DUE_DATE'])
df


,JOB,PROCESSING_TIME,DUE_DATE
0,1,33,35
1,2,17,90
2,3,6,43
3,4,40,85
4,5,22,60
5,6,8,23
6,7,27,76
7,8,19,59
8,9,30,99


# Calculate number of days late, don't count the early

In [3]:
def days_late(df):
    # this assumes that the data frame is in order and done_date and Days_late are correct!!!!
    filter = df['days_late'] > 0
    return df[filter]['days_late'].sum()

# Recalculate the cummulative times based 
# on the sorted order of the data frame

In [4]:
def recalculate_time(df):
    df['done_date'] = df.PROCESSING_TIME.cumsum()
    df['days_late'] = df['done_date'] - df['DUE_DATE']
    return df

# If run in the given job order

In [5]:
df = recalculate_time(df)
print('Days late total:',days_late(df))
df

Days late total: 478


,JOB,PROCESSING_TIME,DUE_DATE,done_date,days_late
0,1,33,35,33,-2
1,2,17,90,50,-40
2,3,6,43,56,13
3,4,40,85,96,11
4,5,22,60,118,58
5,6,8,23,126,103
6,7,27,76,153,77
7,8,19,59,172,113
8,9,30,99,202,103


# Initial Sort by PROCESSING_TIME

In [6]:
df = df.sort_values(by='PROCESSING_TIME').reset_index(drop=True)
df = recalculate_time(df)
print('Days late total:',days_late(df))
df

Days late total: 309


,JOB,PROCESSING_TIME,DUE_DATE,done_date,days_late
0,3,6,43,6,-37
1,6,8,23,14,-9
2,2,17,90,31,-59
3,8,19,59,50,-9
4,5,22,60,72,12
5,7,27,76,99,23
6,9,30,99,129,30
7,1,33,35,162,127
8,4,40,85,202,117


# Split the sorted list at the cummulative done_date by the last due date


In [7]:
last_due_date = df.DUE_DATE.max()
last_due_date

99

# Create the split filter

In [8]:
filter = df['done_date'] <= last_due_date

# These are the jobs that finish before the last DUE_DATE

In [9]:
df[filter]

,JOB,PROCESSING_TIME,DUE_DATE,done_date,days_late
0,3,6,43,6,-37
1,6,8,23,14,-9
2,2,17,90,31,-59
3,8,19,59,50,-9
4,5,22,60,72,12
5,7,27,76,99,23


# These are the jobs that will finish after the last DUE_DATE

In [10]:
df[~filter]

,JOB,PROCESSING_TIME,DUE_DATE,done_date,days_late
6,9,30,99,129,30
7,1,33,35,162,127
8,4,40,85,202,117


# Sort the jobs that can get done before time is up by the DUE_DATE
##### This is to maximize the number of jobs that finish on time (no penelty) and minimize the over due counts

In [11]:
df_A = df[filter].copy()
df_A = df_A.sort_values(by='DUE_DATE').reset_index(drop=True)
df_A = recalculate_time(df_A)
print('Days late total:',days_late(df_A))
df_A

Days late total: 15


,JOB,PROCESSING_TIME,DUE_DATE,done_date,days_late
0,6,8,23,8,-15
1,3,6,43,14,-29
2,8,19,59,33,-26
3,5,22,60,55,-5
4,7,27,76,82,6
5,2,17,90,99,9


# Sort the jobs that will finish after the time is up by PROCESSING_TIME
##### This is to minimize the number of unfinished jobs per day after the last deadline

In [12]:
df_B = df[~filter].copy()
df_B = df_B.sort_values(by='PROCESSING_TIME').reset_index(drop=True)
df_B

,JOB,PROCESSING_TIME,DUE_DATE,done_date,days_late
0,9,30,99,129,30
1,1,33,35,162,127
2,4,40,85,202,117


In [13]:
df_result = pd.concat([df_A,df_B]).reset_index(drop=True)
df_result = recalculate_time(df_result)
print('Days late total:',days_late(df_result))
df_result

Days late total: 289


,JOB,PROCESSING_TIME,DUE_DATE,done_date,days_late
0,6,8,23,8,-15
1,3,6,43,14,-29
2,8,19,59,33,-26
3,5,22,60,55,-5
4,7,27,76,82,6
5,2,17,90,99,9
6,9,30,99,129,30
7,1,33,35,162,127
8,4,40,85,202,117
